In [1]:
from trainer import *

model_filename = 'models/char_rnn_20_epochs_1562264474'
model = load_model(model_filename)

### `predict()` 

So first let's look at `predict()` function. Provided some input character it generates some output.

In [19]:
h = model.init_hidden(1)
char, h = predict(model, char='H', h=h, top_k=5)
print(char)

o


(1) First we have to one-hot encoded our input char.

In [53]:
char = 'X'
x = np.array([[model.char2int[char]]])
i = x.item()
x, x.shape, i

(array([[65]]), (1, 1), 65)

In [54]:
x = one_hot_encode(x, len(model.chars))
print(x, x.shape, x[:, :, i])
inputs = torch.from_numpy(x)

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]] (1, 1, 83) [[1.]]


(2) Now we're making forward pass to get logits and apply `softmax` to get probs.

In [55]:
h = tuple([each.data for each in h])
out, h = model(inputs, h)

In [56]:
out.shape

torch.Size([1, 83])

In [72]:
p = F.softmax(out, dim=1).data

In [73]:
p.shape, torch.sum(p, dim=1), p[:, :5]

(torch.Size([1, 83]),
 tensor([1.0000]),
 tensor([[0.0002, 0.0113, 0.0002, 0.0430, 0.0066]]))

(3) Next step is to apply `topk()`. First we choose 5 chars with highest probability and then pick one of them randomply. This is a trick that is described in the text of the assignment.

In [74]:
top_k=5
top_p, top_ch = p.topk(top_k)
top_p, top_ch

(tensor([[0.2792, 0.1329, 0.0660, 0.0497, 0.0493]]),
 tensor([[68, 77, 18, 35, 38]]))

In [75]:
top_ch = top_ch.numpy().squeeze()
top_ch

array([68, 77, 18, 35, 38])

In [76]:
top_p = top_p.numpy().squeeze()
char = np.random.choice(top_ch, p=top_p / top_p.sum())
char

18

### `sample()`

And now the second function that generates text of the given size. For example in the assignment we generate `1000` chars. Traditional sampling is based on empty first character. But here we use initial word. Eventually we produce a char and use as an input for the next step.

In [78]:
sample(model, size=25, prime='The', top_k=5)

'Ther at the\nbook. I should no'

So here's the question: how can we feed to our model a single char instead of a sequence? That's not quite clear. But to answer this question we have to look closer at shapes of our input and output.